# Spark Data Wrangling

In [29]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from env import user, pwd, host
import os

In [2]:
def get_db_url(db):
    '''input df and output sql connection string'''
    return (f'mysql+pymysql://{user}:{pwd}@{host}/{db}')

## Acquire

In [3]:
#create enviroment
spark = SparkSession.builder.getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/19 14:53:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### load mpg data set from pydataset

In [4]:
from pydataset import data

In [5]:
mpg = spark.createDataFrame(data('mpg'))
mpg

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

In [6]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



### write datafame to file

- `json`: for writing to a local json file(s)
- `csv`: for writing to a local csv file(s)
- `parquet`: Parquet is a very popular columnar storage format for Hadoop.
- `jdbc`: for writing to a SQL database table

#### write file to json

In [7]:
#df.write.type
mpg.write.json('data/mpg_json',mode='overwrite')

In [8]:
import os

In [9]:
os.listdir('data/mpg_json/')

['part-00005-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '.part-00006-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json.crc',
 'part-00000-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '.part-00005-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json.crc',
 'part-00002-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '._SUCCESS.crc',
 '.part-00002-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json.crc',
 'part-00007-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '.part-00001-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json.crc',
 'part-00006-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '.part-00007-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json.crc',
 'part-00003-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '.part-00004-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json.crc',
 '_SUCCESS',
 'part-00001-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '.part-00003-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json.crc',
 'part-00004-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json',
 '.p

In [10]:
first_file = [fn for fn in os.listdir('data/mpg_json/') if not fn.startswith('.')][0]

In [11]:
first_file

'part-00005-61804ae3-1ce4-4925-b047-5a9521172f6e-c000.json'

In [12]:
spark.read.json(f'data/mpg_json/{first_file}').count()

29

In [13]:
spark.read.json(f'data/mpg_json/').count()

234

#### write dataframe to csv

In [14]:
#df.write.format()
(
mpg.write.format('csv')
    .mode('overwrite')
    .option('header',True)
    .save('data/mpg_csv')

)

### read files
- spark.read.[type]

#### read json

#### read csv

In [15]:
#keep written csv headers
(
    spark.read.format("csv")
      .option("header", True)
      .load("data/mpg_csv")
).count()

234

In [16]:
#keep written csv headers
(
    spark.read.format("csv")
      .option("header", True)
      .load("data/mpg_csv")
)

DataFrame[manufacturer: string, model: string, displ: string, year: string, cyl: string, trans: string, drv: string, cty: string, hwy: string, fl: string, class: string]

### load source from 311_data in sql

In [17]:
# #sql query
# url = get_db_url('311_data')
# query = 'select source_id, source_username from source'

In [18]:
# #make pandas df
# pandas_df = pd.read_sql(query, url)
# pandas_df.head()

In [19]:
pandas_df = pd.read_csv('311.csv')

### load cases from 311_data from sql

In [20]:
# #sql query
# query = 'select * from cases limit 100000'

In [21]:
#pandas df
# pandas_df = pd.read_sql(query, url)

In [57]:
#spark df
df = spark.createDataFrame(pandas_df)
df

DataFrame[case_id: bigint, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: bigint]

In [23]:
df.show(3, vertical=True, truncate=False)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616                          
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

23/05/19 14:54:09 WARN TaskSetManager: Stage 16 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


## Prepare

- rename columns
- correct datatypes
- data transformation
- make new features
- join tables

### rename columns

#### change SLA_due_date to case_due_date

In [ ]:
df = df.withColumnRenamed('SLA_due_date','case_due_date')

### correct datatypes

In [58]:
df.select('case_closed','case_late').show(1)

23/05/19 16:26:02 WARN TaskSetManager: Stage 27 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|        YES|       NO|
+-----------+---------+
only showing top 1 row



#### change close_closed and case_late columns into boolean values

In [ ]:
#use condition to make true and false


#### change council_district datatype to string

In [24]:
#use .cast()
df.select('council_district').show(5)

23/05/19 15:02:23 WARN TaskSetManager: Stage 17 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
|               3|
|               7|
+----------------+
only showing top 5 rows



In [32]:
df.withColumn('council_district',
             F.col('council_district').cast('string')).show(5)

+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO| -2.012604167|        YES|     Storm Water|Removal Of Obstru...|4.322222222|     Closed| svcCRMSS|2215  GOL

23/05/19 15:05:05 WARN TaskSetManager: Stage 18 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


#### change dates to datetype

format date strings: https://docs.oracle.com/javase/10/docs/api/java/time/format/DateTimeFormatter.html

In [35]:
fmt = 'M/d/yy H:m'

In [36]:
#use to_timestamp
df.select(
    F.to_timestamp('case_opened_date',fmt),
    F.to_timestamp('case_closed_date',fmt)

).show(2)

+------------------------------------------+------------------------------------------+
|to_timestamp(case_opened_date, M/d/yy H:m)|to_timestamp(case_closed_date, M/d/yy H:m)|
+------------------------------------------+------------------------------------------+
|                       2018-01-01 00:42:00|                       2018-01-01 12:29:00|
|                       2018-01-01 00:46:00|                       2018-01-03 08:11:00|
+------------------------------------------+------------------------------------------+
only showing top 2 rows



23/05/19 15:09:34 WARN TaskSetManager: Stage 20 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [40]:
#use to_timestamp
df.withColumn('case_opened_date',
    F.to_timestamp('case_opened_date',fmt)).withColumn('case_closed_date',
    F.to_timestamp('case_closed_date',fmt))

DataFrame[case_id: bigint, case_opened_date: timestamp, case_closed_date: timestamp, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: bigint]

### data transformation

#### normalize address
- `lower`: lowercase everything
- `trim`: remove whitespace on the edges 

In [42]:
df.select(
    F.trim(F.lower('request_address'))
).show()

+----------------------------+
|trim(lower(request_address))|
+----------------------------+
|        2315  el paso st,...|
|        2215  goliad rd, ...|
|        102  palfrey st w...|
|        114  la garde st,...|
|        734  clearview dr...|
|        bandera rd and br...|
|        10133  figaro can...|
|        10133  figaro can...|
|        10133  figaro can...|
|        10133  figaro can...|
|        10133  figaro can...|
|        10133  figaro can...|
|        10129  boxing pas...|
|        10129  boxing pas...|
|        10129  boxing pas...|
|        834  barrel point...|
|        834  barrel point...|
|        834  barrel point...|
|        834  barrel point...|
|        834  barrel point...|
+----------------------------+
only showing top 20 rows



23/05/19 15:12:20 WARN TaskSetManager: Stage 23 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [43]:
df.withColumn('request_address',
    F.trim(F.lower('request_address'))
)

DataFrame[case_id: bigint, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: bigint]

#### change num_days_late to num_weeks_late

In [44]:
df.select(
'num_days_late'

).show(5)

+-------------+
|num_days_late|
+-------------+
| -998.5087616|
| -2.012604167|
| -3.022337963|
| -15.01148148|
|  0.372164352|
+-------------+
only showing top 5 rows



23/05/19 15:13:39 WARN TaskSetManager: Stage 24 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [49]:
df.select(
    F.expr('num_days_late/7')).show(5)

+--------------------+
| (num_days_late / 7)|
+--------------------+
|        -142.6441088|
|        -0.287514881|
|-0.43176256614285713|
| -2.1444973542857144|
|0.053166335999999995|
+--------------------+
only showing top 5 rows



23/05/19 15:14:38 WARN TaskSetManager: Stage 25 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [51]:
df.withColumn(
    'num_weeks_late',
    F.expr('num_days_late/7'))

DataFrame[case_id: bigint, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: bigint, num_weeks_late: double]

#### change council_district to int and pad with 00s

In [56]:
F.concat(F.lit('00'), F.col('council_district')).show(5)

TypeError: 'Column' object is not callable

### new features

#### create zip code column

#### create case_lifetime column

- case_age: how long since the case first opened
- days_to_close: the number of days between days opened and days closed
- case_lifetime: if the case is open, how long since the case opened, if the case is closed, the number of days to close


In [ ]:
#use datediff() to find the difference between two dates


In [ ]:
#create case_lifetime column


In [ ]:
#drop unnecessary columns


### join the dept table from sql to our current df

In [ ]:
df.select('dept_division').show(5)

In [ ]:
#get dept table from sql
query = 'select * from dept'

In [ ]:
url = get_db_url('311_data')
dept = pd.read_sql(query, url)

In [ ]:
dept = spark.createDataFrame(dept)
dept

### train, validate, test split

- `.randomSplit` to split df